In [3]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F
import kagglehub

# Download latest version
path = kagglehub.dataset_download("hojjatk/mnist-dataset")

print("Path to dataset files:", path)

/Users/kartikmehta/.pyenv/versions/3.11.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 22.0M/22.0M [00:01<00:00, 12.7MB/s]

Extracting files...


Path to dataset files: /Users/kartikmehta/.cache/kagglehub/datasets/hojjatk/mnist-dataset/versions/1


In [11]:
# read files

import numpy as np

def read_idx1_labels(filename):
    with open(filename, 'rb') as f:
        magic = int.from_bytes(f.read(4), 'big')
        num_labels = int.from_bytes(f.read(4), 'big')
        label_data = np.frombuffer(f.read(), dtype=np.uint8)
    return label_data

def read_idx3(filename):
    with open(filename, 'rb') as f:
        magic = int.from_bytes(f.read(4), 'big')
        num_images = int.from_bytes(f.read(4), 'big')
        rows = int.from_bytes(f.read(4), 'big')
        cols = int.from_bytes(f.read(4), 'big')
        image_data = np.frombuffer(f.read(), dtype=np.uint8)
        images = image_data.reshape(num_images, rows, cols)
    return images

train_x = read_idx3('../data/mnist/train-images.idx3-ubyte')
train_y = read_idx1_labels('../data/mnist/train-labels.idx1-ubyte')

print(f"Train X Shape: {train_x.shape}")
print(f"Train Y Shape: {train_y.shape}")

Train X Shape: (60000, 28, 28)
Train Y Shape: (60000,)


In [14]:
train_x = torch.tensor(train_x).float()
train_y = torch.tensor(train_y).float()

/var/folders/71/tcj0hk9j1936q04nz4tmvr2m0000gn/T/ipykernel_21437/944314622.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_x = torch.tensor(train_x).float()
/var/folders/71/tcj0hk9j1936q04nz4tmvr2m0000gn/T/ipykernel_21437/944314622.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_y = torch.tensor(train_y).float()


In [ ]:
class MyNet(nn.Module):
    def __init__(self):
        self.l1 = nn.Linear(28*28, 256)
        self.l2 = nn.Linear(256, 128)
        self.l3 = nn.Linear(128, 64)
        self.l4 = nn.Linear(64, 10)
    
    def forward(self, x):
        x = F.relu(self.l3(F.relu(self.l2(self.l1(x)))))
        x = F.softmax(self.l4(x))
        
        return x

In [ ]:
import random as rand

epochs = 1
batch_size = 64

model = MyNet()

for _ in range(epochs):
    print(f"Epoch: {epochs}")

    start_ix = rand.randint(0, train_x.shape[0]-batch_size)
    inp = train_x[start_ix:start_ix+batch_size]
    out = model(inp)
    print(out.shape)